In [17]:
import json

In [18]:
def extract_parameters(schema):
    """ Extract parameters from the schema into a list of dicts suitable for appArgs. """
    app_description = schema["description"]
    defs = schema["definitions"]
    
    # Sections to skip
    skip_sections = {
        "pipeline_report",
        "generic_options",
        "max_job_request_options",
        "institutional_config_options"
    }

    app_args = []
    for definition_key, section_value in defs.items():
        if definition_key in skip_sections:
            continue  

        properties = section_value.get("properties", {})
        for param, details in properties.items():
            # Check if a default value is provided and include it in the arg string
            if "default" in details:
                arg_value = f"--{param} {details['default']}"
            else:
                arg_value = f"--{param}"

            if details.get("type") == "boolean":
                input_mode = "INCLUDE_ON_DEMAND"
            else:
                input_mode = "REQUIRED"

            arg_entry = {
                "name": param,
                "arg": arg_value,
                "description": details.get("description", ""),
                "inputMode": input_mode
            }
            app_args.append(arg_entry)
    return app_description, app_args


In [19]:
file_path = './nextflow_schema.json'

with open(file_path, 'r') as file:
    schema = json.load(file)

app_description, app_args = extract_parameters(schema)

### Application definition template

In [20]:
app_name = "ampliseq"
app_id_hpc = "ampliseq_pipeline_test"
version = "0.1"
user = "andyyu"
system_id_hpc = "test-zip-koa-hpc-andyyu"
memory_mb = 16000
max_minutes = 30

In [21]:
app_def_hpc= {
    "id": app_name,
    "version": version,
    "description": app_description,
    "runtime": "ZIP",
    "runtimeOptions": ["NONE"],
    "strictFileInputs": False,
    "containerImage": f"/home/{user}/koa_scratch/apps/{app_id_hpc}-pipeline-app-v{version}.tar.gz",
    "jobType": "BATCH",
    "jobAttributes": {
        "archiveOnAppError": True,
        "archiveSystemId": system_id_hpc,
        "archiveSystemDir": "${JobWorkingDir}/jobs/${JobUUID}",
        "execSystemId": system_id_hpc,
        "parameterSet": {
            "archiveFilter": {
                "includeLaunchFiles": False
            },
            "appArgs": app_args

        },
        "memoryMB": memory_mb,
        "nodeCount": 1,
        "coresPerNode": 1,
        "maxMinutes": max_minutes
    },
}